In [1]:
WORD_NUM = 5 # MNRead has 15 words at most per line
penalty = 0.02

# Load in Data

In [2]:
import json
from pathlib import Path
import pandas as pd
import ast
from collections import Counter
from tqdm import tqdm

In [3]:
def count_matches_per_row(row):
    left_side = pd.Series(row['text'].lower().split(' '))
    right_side = pd.Series([i.lower() for i in row['rec_texts']])
    
    counter_left = Counter(left_side.dropna()) # dic
    counter_right = Counter(right_side.dropna())
    
    # 计算两侧相同元素的个数，确保每个元素只计算一次
    matches = sum((counter_left & counter_right).values())
    total = sum(counter_left.values())
    
    # char_level
    counter_left_char = Counter(''.join(left_side.dropna().tolist()))
    counter_right_char = Counter(''.join(right_side.dropna().tolist()))
    matches_char = sum((counter_left_char & counter_right_char).values())
    total_char = sum(counter_left_char.values())
    return [matches, total - matches, total_char-matches_char]


In [4]:
filter_df = pd.read_csv('../../data/human/SelectedFilter.csv')

In [5]:
output_dir = Path("../../filtered/etdrs")
gt_dir = Path("../../data/etdrs")
files = list(output_dir.rglob('*.*'))
result_li = []
ba_li = []
df_human = pd.read_csv('../../data/human/human_etdrs_acuity.csv')
df_human.rename(columns={'acuity': 'human'}, inplace=True)
# Load the ground truth COCO JSON (annotations only or full dataset)
with open(gt_dir/"anno.json", "r") as f:
    ground_truth = json.load(f)
# Here we assume ground_truth has a key "annotations"
gt_annotations = ground_truth.get("annotations", [])
img_info = ground_truth.get("images", [])

for model_file in tqdm(files, total=len(files), desc="Rinse model outputs"):
    # Load the model output JSON (replace with your file or object)
    # model_file = output_dir/ "gpt4o.json"
    model_name = model_file.stem
    with open(model_file, "r") as f:
        model_output = json.load(f)

    # Create lookup dictionaries for ground truth annotations and image info by image_id.
    # If there are multiple ground truth annotations per image, you might need to store a list.
    gt_by_chart = {}
    for info, ann in zip(img_info, gt_annotations):
        id = ann["id"]
        full = {}
        full["image_id"] = ann["image_id"]
        full["file_name"] = info["file_name"]
        full["Filter_no"] = info["Filter_no"]
        full["text"] = ann["caption"]
        gt_by_chart.setdefault(id, full)
        
    
    # Now iterate through the model outputs and match with ground truth and image info
    full_out = {}
    for output_item in model_output:
        image_id = output_item["image_id"]
        
        # Optionally, if your model output's "rec_texts" is a string representation of a list,
        # convert it to an actual list.
        rec_texts_str = output_item.get("rec_texts", "")
        try:
            rec_texts = ast.literal_eval(rec_texts_str)
        except Exception:
            rec_texts = rec_texts_str
        output_item["rec_texts"] = rec_texts

        # Retrieve corresponding ground truth annotations (if any)
        gt_matches = gt_by_chart.get(image_id, None)
        if gt_matches is None:
            continue
        chart_no, reso, row_no = Path(gt_matches.get("file_name", " _ _ _")).stem.split("_")
        full_out[image_id] = gt_matches
        full_out[image_id]["rec_texts"] = rec_texts
        full_out[image_id]["chart_no"] = chart_no
        full_out[image_id]["reso"] = reso
        full_out[image_id]["row_no"] = row_no
        
        
    df = pd.DataFrame(full_out.values())
    df = pd.merge(df, filter_df, on='Filter_no', how='inner')

    df[['match','missing','missing_char']] = df.apply(lambda row: count_matches_per_row(row), axis=1).apply(pd.Series)
    df['missing_clipped'] = df['missing'].clip(upper=5)

    group_sum = df.groupby(['chart_no','a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: -0.2 + x['missing_clipped'].sum()*penalty).reset_index()
    group_sum.columns = ['chart_no', 'a', 'b', 'VA', 'CS', 'Cond','acuity']
    group_sum = group_sum.groupby(['a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: x['acuity'].mean()).reset_index()
    group_sum.columns = ['a', 'b', 'VA', 'CS', 'Cond',model_name]
    result_li.append(group_sum)

    # BA-test
    df_ba = df.copy().groupby(['chart_no','a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: -0.2 + x['missing_clipped'].sum()*penalty).reset_index()
    df_ba.columns = ['chart_no', 'a', 'b', 'VA', 'CS', 'Cond','acuity']
    df_ba = pd.merge(df_ba,df_human, on=['a', 'b'],how='left')
    results = []

    for cond, group in df_ba.groupby(['a', 'b', 'VA', 'CS', 'Cond']):
        diff = (group['acuity'] - group['human'])
        avg = (group['acuity'] + group['human'])/2
        bias = diff.mean()
        loa_low = bias - 1.96 * diff.std()
        loa_high = bias + 1.96 * diff.std()
            
        results.append({
                'Expected': cond[2], # VA
                'Cond': cond[4],
                'Model': model_name,
                'Bias': bias,
                'Diff': diff,
                'Avg': avg,
                'LoA_Low': loa_low,
                'LoA_High': loa_high
            })
    ba_li.append(pd.DataFrame(results))


Rinse model outputs:   0%|          | 0/22 [00:00<?, ?it/s]

/tmp/ipykernel_3436723/2929039427.py:67: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_sum = df.groupby(['chart_no','a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: -0.2 + x['missing_clipped'].sum()*penalty).reset_index()
/tmp/ipykernel_3436723/2929039427.py:69: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_sum = group_sum.groupby(['a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: x['acuity'].mean())

In [7]:
from functools import reduce
summary_dir = Path("../../data/summary")
# reduce(lambda left, right: pd.merge(left, right, on=['a', 'b', 'VA', 'CS', 'Cond']), result_li).to_csv(summary_dir/'etdrs.csv', index=False)

In [6]:
import pandas as pd
from functools import reduce
df1 = reduce(lambda left, right: pd.merge(left, right, on=['a', 'b', 'VA', 'CS', 'Cond']), result_li)
df2 = pd.read_csv(summary_dir/'LetterSummary_0920.csv')
df2['a'] = df2['a'].round(3)
df2['b'] = df2['b'].round(3)
df2['a'].replace(0.156, 0.157, inplace=True)
df2['b'].replace(0.156, 0.157, inplace=True)
df2['a'].replace(0.287, 0.288, inplace=True)
df2['b'].replace(0.287, 0.288, inplace=True)
df2.drop(columns=['image_no','Filter_no','human'], inplace=True)
df2 = df2.groupby(['a', 'b']).mean().reset_index()

df_human = pd.read_csv('../../data/human/human_etdrs_acuity.csv')
df_human = df_human.rename(columns={'acuity': 'human'})

NameError: name 'summary_dir' is not defined

In [9]:
final_df = pd.merge(pd.merge(df1, df_human, on=['a', 'b'],how='left'), df2, on=['a', 'b'], how='left')
models = ['human','gpt4o', 'gpt4o_mini', 'gemini_15_flash',
       'gemini_15_pro', 'gemini_2_flash', 'claude3_7_sonnet', 'claude3_5_haiku','cogvlm', 'Qwen2.5-VL-3B-Instruct',
       'Qwen2.5-VL-7B-Instruct', 'Qwen2.5-VL-32B-Instruct', 'qwen', 'maerec','spts', 'ppocr', 'azure', 'google',
       'gemini-2.5-flash', 'gemini-2.5-pro', 'gpt-5-mini', 'gpt-5', 'DeepSeek-OCR_Small',
       'DeepSeek-OCR_Base', 'DeepSeek-OCR_Tiny', 'DeepSeek-OCR_Large',
       'DeepSeek-OCR_Gundam','seeingai']
final_df = final_df[['a', 'b', 'VA', 'CS', 'Cond', 'Expected'] + models]
# final_df.to_csv(summary_dir/'etdrs_combined_1021.csv', index=False)

# Figure

In [8]:
import pandas as pd
final_df_fig = pd.read_csv('../../data/summary/etdrs_combined_1021.csv')#final_df.copy()
models = ['human','SeeingAI', 'gpt4o', 'gpt4o_mini', #'gemini_15_flash','gemini_15_pro',
        'gemini_2_flash', 'claude3_7_sonnet', 'claude3_5_haiku',
       'cogvlm','Qwen2.5-VL-3B-Instruct', 'Qwen2.5-VL-7B-Instruct', 'Qwen2.5-VL-32B-Instruct',
       'maerec','spts', 'google', 'gemini-2.5-flash', 'gemini-2.5-pro', 'gpt-5-mini', 'gpt-5', 'DeepSeek-OCR_Small',
       'DeepSeek-OCR_Base', 'DeepSeek-OCR_Tiny', 'DeepSeek-OCR_Large',
       'DeepSeek-OCR_Gundam'] # 'ppocr', 'azure',
# models = ['human','gemini-2.5-flash', 'gemini-2.5-pro','gpt-5','gpt-5-mini'] #word

# comment this line to keep the original data for BA
final_df_fig[['Expected'] + models] =  final_df_fig[['Expected'] + models]  - final_df_fig.loc[15,['Expected'] + models]
final_df_fig.loc[final_df_fig['Cond']=='Original', 'Cond'] = 'Combined'

model_name_dict = {'human':'Human','SeeingAI':'SeeingAI', 'gpt4o':'GPT4O', 'gpt4o_mini':'GPT4O Mini', 
                   'gemini_15_flash':'Gemini-1.5 Flash','gemini_15_pro':'Gemini-1.5 Pro', 'gemini_2_flash':'Gemini-2 Flash', 
                   'claude3_7_sonnet':'Claude3.7 Sonnet', 'claude3_5_haiku':'Claude3.5 Haiku','cogvlm':'CogAgent',
                   'Qwen2.5-VL-3B-Instruct':'Qwen2.5-VL-3B', 'Qwen2.5-VL-7B-Instruct':'Qwen2.5-VL-7B', 'Qwen2.5-VL-32B-Instruct':'Qwen2.5-VL-32B',
                   'maerec':'DBNet++ & MaeRec','spts':'SPTS v2', 'ppocr':'PPOCR', 'azure':'Azure', 'google':'Google Vision',
                   'gemini-2.5-flash':'Gemini-2.5 Flash','gemini-2.5-pro':'Gemini-2.5 Pro', 'gpt-5-mini':'GPT-5 Mini', 'gpt-5':'GPT-5', 'DeepSeek-OCR_Small':'DeepSeek-OCR Small',
                   'DeepSeek-OCR_Base':'DeepSeek-OCR Base', 'DeepSeek-OCR_Tiny':'DeepSeek-OCR Tiny', 'DeepSeek-OCR_Large':'DeepSeek-OCR Large',
                   'DeepSeek-OCR_Gundam':'DeepSeek-OCR Gundam', 'seeingai':'SeeingAI'}
final_df_fig.rename(columns=model_name_dict, inplace=True)
models = [model_name_dict[i] for i in models if i in model_name_dict]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import numpy as np
sns.set_theme(style="white")
# Define a function to choose marker based on condition
def get_marker(cond):
    if cond == 'Horizontal':
        return 'o'  # hollow square
    elif cond == 'Vertical':
        return '^'  # hollow triangle
    else:
        return 's'  # hollow circle

# Create a color palette for unique conditions
unique_conditions = final_df_fig['Cond'].unique()
# palette = sns.color_palette("viridis", len(unique_conditions))
# color_mapping = {cond: palette[i] for i, cond in enumerate(unique_conditions)}
colors = ['orange', 'black', '#6A9C89', 'black']
# Create the mapping
color_mapping = {cond: color for cond, color in zip(unique_conditions, colors)}

# Define a function to plot a scatter plot on a given axis for a specified model column
def plot_scatter(ax, model_col, df):
    # Add a dashed diagonal line that spans the range of the data
    x_vals = df['Expected']
    y_vals = df[model_col]
    overall_min = -0.1#min(x_vals.min(), y_vals.min())
    overall_max = 2.0#max(x_vals.max(), y_vals.max())
    ax.plot([overall_min, overall_max], [overall_min, overall_max],
            linestyle='--', color='grey', linewidth=2.5)
    x = np.linspace(overall_min, overall_max, 100)
    # Fill the area between y = x - 0.2 and y = x + 0.2
    ax.fill_between(x, x - 0.2, x + 0.2, color='grey', alpha=0.25)
    
    # Plot each group (by Cond) separately to assign markers and colors
    for cond, group in df.groupby('Cond'):
        marker = get_marker(cond)
        ax.scatter(
            group['Expected'],
            group[model_col],
            label=cond,
            marker=marker,
            facecolors=color_mapping[cond],  # makes marker hollow
            edgecolors=color_mapping[cond],
            s=100  # marker size
        )
        
    # calulate the correlation
    from scipy.stats import linregress
    slope, intercept, r_value, p_value, std_err = linregress(x_vals.astype(float).to_numpy(), y_vals.astype(float).to_numpy())
    r_squared = r_value ** 2
    # Annotate R² and p-value in bottom-right
    ax.text(0.96, 0.02, f'$R^2$ = {r_squared:.2f}, '+ (f'p = {p_value:.3f}' if p_value > 0.001 else 'p < 0.001'),
            ha='right', va='bottom', transform=ax.transAxes,
            fontsize=11)

    
    # Set labels, title and legend
    ax.set_xlabel("Expected Acuity Change (logMAR)")
    ax.set_ylabel("Measured Acuity Change (logMAR)")
    ax.set_xlim(-0.1, 2.0)
    ax.set_ylim(-0.1, 2.0)
    ax.set_title(f"{model_col}", fontsize=16)
    # ax.legend()

# List of models to plot (include 'human' and other models)
# models = ['human','SeeingAI', 'gpt4o', 'gpt4o_mini', 'gemini_15_flash',
#        'gemini_15_pro', 'gemini_2_flash', 'claude3_7_sonnet', 'claude3_5_haiku',
#        'cogvlm','Qwen2.5-VL-3B-Instruct', 'Qwen2.5-VL-7B-Instruct', 'Qwen2.5-VL-32B-Instruct',
#        'qwen', 'maerec','spts', 'ppocr', 'azure', 'google']

# Determine subplot grid layout; here we choose 2 columns
num_models = len(models)
ncols = 5
nrows = math.ceil(num_models / ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(3 * ncols, 3 * nrows))
# If there's only one subplot, axes is not an array; make it a list for consistency.
if num_models == 1:
    axes = [axes]
else:
    axes = axes.flatten()

# Plot each model in its corresponding subplot
for i, model in enumerate(models):
    plot_scatter(axes[i], model, final_df_fig)

# Remove any extra subplots if the grid has more axes than models
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])
    
handles, labels = axes.flatten()[0].get_legend_handles_labels()

# Place a common legend outside the figure. Here, loc='upper center' puts it at the top,
# and bbox_to_anchor adjusts its position. Adjust ncol for number of columns in legend.
# fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=3, fontsize=14)
plt.tight_layout()
plt.savefig('../../figure/etdrs.pdf', dpi=300)

# # Plot for 'human'
# for model in models:
#     plot_scatter(model, final_df_fig)

In [ ]:
def rmse(arr):
    return np.sqrt(np.mean(arr ** 2))
std_values = final_df_fig[models].sub(final_df_fig['Expected'], axis=0).apply(rmse, axis=0)
# Identify the model with the smallest std
min_model = std_values.idxmin()

# Create a color list: red for the model with the smallest std; use a nice color (e.g. 'skyblue') for others.
colors = ['#D76C82' if model == min_model else 'gray' for model in std_values.index]

# Optional: sort the values for a prettier display (and sort colors accordingly)
std_values_sorted = std_values.sort_values(ascending=False)

colors_sorted = [colors[std_values.index.get_loc(model)] for model in std_values_sorted.index]
# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
std_values_sorted.plot(kind='bar', color=colors_sorted, edgecolor='none', width=0.9, ax=ax)

for patch, tick in zip(ax.patches, ax.get_xticklabels()):
    if tick.get_text().lower() == 'human':  # adjust case if needed
        # Set no fill (hollow)
        patch.set_facecolor('none')
        # Set dashed edge
        patch.set_linestyle('--')
        patch.set_linewidth(2)
        # Optionally, set a specific edge color. For instance, if you want red:
        patch.set_edgecolor('#D76C82')
# Customize axis labels, title, and tick parameters
# plt.xlabel("Model", fontsize=14, fontweight='bold')
plt.ylabel("Root Mean Square Error", fontsize=14)
plt.yticks(np.arange(0,1.01,0.25),fontsize=12)
plt.xticks(fontsize=12)
plt.tight_layout()
# plt.savefig('../../figure/etdrs_rmse.pdf', dpi=300)


In [ ]:
def rmse(arr):
    return np.sqrt(np.mean(arr ** 2))
std_values = final_df_fig[models[1:]].sub(final_df_fig['Human'], axis=0).apply(rmse, axis=0)
# Identify the model with the smallest std
min_model = std_values.idxmin()

# Create a color list: red for the model with the smallest std; use a nice color (e.g. 'skyblue') for others.
colors = ['green' if model == min_model else 'gray' for model in std_values.index]

# Optional: sort the values for a prettier display (and sort colors accordingly)
std_values_sorted = std_values.sort_values(ascending=False)

colors_sorted = [colors[std_values.index.get_loc(model)] for model in std_values_sorted.index]
# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
std_values_sorted.plot(kind='bar', color=colors_sorted, edgecolor='none', width=0.9, ax=ax)

# Customize axis labels, title, and tick parameters
# plt.xlabel("Model", fontsize=14, fontweight='bold')
plt.ylabel("Root Mean Square Error", fontsize=14)
plt.yticks(np.arange(0,1.01,0.25),fontsize=12)
plt.xticks(rotation=75, fontsize=12)
plt.tight_layout()
plt.savefig('../../figure/etdrs_rmse_tohuman.pdf', dpi=300)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

def rmse(arr):
    return np.sqrt(np.mean(arr ** 2))

# --- metrics ---
rmse_vs_exp = final_df_fig[models].sub(final_df_fig['Expected'], axis=0).apply(rmse, axis=0)
models_wo_human = [m for m in models if str(m).casefold() != 'human']
rmse_to_human = final_df_fig[models_wo_human].sub(final_df_fig['Human'], axis=0).apply(rmse, axis=0)

# --- order & align (ascending by left) ---
order = rmse_vs_exp.sort_values(ascending=True).index.tolist()
left_vals  = rmse_vs_exp.reindex(order).values
right_vals = rmse_to_human.reindex(order).values

y = np.arange(len(order))
bar_h = 0.58

# --- colors & highlights ---
min_left_label  = order[np.nanargmin(left_vals)]
min_right_label = rmse_to_human.idxmin() if len(rmse_to_human) else None
left_colors  = ['#D76C82' if lab == min_left_label else '#9AA0A6' for lab in order]
right_colors = []
for lab, v in zip(order, right_vals):
    if np.isnan(v) or str(lab).casefold() == 'human':
        right_colors.append('none')
    elif lab == min_right_label:
        right_colors.append('#2E7D32')
    else:
        right_colors.append('#C2A68C')

# --- axis range & central gap ---
lim_left  = float(np.nanmax(left_vals))
lim_right = float(np.nanmax(right_vals[~np.isnan(right_vals)]) if np.any(~np.isnan(right_vals)) else 0.0)
lim = max(lim_left, lim_right, 1.0)
step = 0.25
max_tick = np.ceil(lim / step) * step

gap = max_tick * 0.22   # 中缝宽度的一半（可调：0.12~0.25）
left_anchor  = -gap      # 左柱起点在 -gap 处向左延伸
right_anchor =  gap      # 右柱起点在 +gap 处向右延伸

fig_h = max(6, 0.45 * len(order) + 2)
fig, ax = plt.subplots(figsize=(12, fig_h))

# 背景/网格
ax.set_facecolor('#FCFCFD')
# ax.grid(axis='x', linestyle='--', linewidth=0.6, alpha=0.5)
for s in ['top', 'right', 'left']: ax.spines[s].set_visible(False)

# --- bars (外移) ---
# 左：从 -gap 向左画，width 用正值，left=-(gap+value)
left_bars = ax.barh(
    y, left_vals, height=bar_h, color=left_colors, edgecolor='none',
    left=-(gap + left_vals), zorder=3
)
# Human 左侧空心
for patch, lab in zip(left_bars, order):
    if str(lab).casefold() == 'human':
        patch.set_facecolor('none'); patch.set_edgecolor('#D76C82')
        patch.set_linestyle('--'); patch.set_linewidth(1.8)

# 右：从 +gap 向右画
right_bars = ax.barh(
    y, np.nan_to_num(right_vals, nan=0.0), height=bar_h,
    color=right_colors, edgecolor='none', left=right_anchor, zorder=3
)
# 隐藏 Human/NaN 右半
for patch, lab, v in zip(right_bars, order, right_vals):
    if np.isnan(v) or str(lab).casefold() == 'human':
        patch.set_visible(False)

# --- 中缝与标题 ---
ax.axvspan(-gap, gap, color='white', alpha=0.95, zorder=1)  # 中间留白带
ax.text(-gap, 1.012, 'vs Expected',            transform=ax.get_xaxis_transform(), ha='right', va='bottom',
        fontsize=12, fontweight='bold', color='#3A3F44')
ax.text( gap, 1.012, 'vs Human',   transform=ax.get_xaxis_transform(), ha='left',  va='bottom',
        fontsize=12, fontweight='bold', color='#3A3F44')

# --- model names 放在中缝里（不与柱子重合） ---
for yi, lab in zip(y, order):
    ax.text(0, yi, str(lab), ha='center', va='center', fontsize=11.2, color='#28323C', zorder=4)

# --- x 轴：对称范围 + 绝对值刻度 ---
xmax = max_tick + gap + max_tick*0.05
ax.set_xlim(-xmax, xmax)
ax.xaxis.set_major_locator(MultipleLocator(step))
ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.set_xticklabels([t.get_text().lstrip('-') for t in ax.get_xticklabels()], fontsize=13)
ax.set_xlabel('Root Mean Square Error', fontsize=15.5, color='#3A3F44')

# 移除 ytick 文本（我们用中缝标签）
ax.set_yticks(y); ax.set_yticklabels([]); ax.tick_params(axis='y', length=0)

# --- 值标签（条末端） ---
pad = max_tick * 0.015
for v, b in zip(left_vals, left_bars):
    if v <= 0: continue
    x = -gap - v - pad
    ax.text(x, b.get_y()+b.get_height()/2, f'{v:.2f}'.rstrip('0').rstrip('.'),
            ha='right', va='center', fontsize=12, color='#4A4F55', zorder=4)
for v, b in zip(right_vals, right_bars):
    if np.isnan(v) or v <= 0: continue
    x =  gap + v + pad
    ax.text(x, b.get_y()+b.get_height()/2, f'{v:.2f}'.rstrip('0').rstrip('.'),
            ha='left', va='center', fontsize=12, color='#4A4F55', zorder=4)

# --- 图例 ---
legend_handles = [
    Patch(facecolor='#D76C82', edgecolor='none', label='Min RMSE (vs Expected)'),
    Patch(facecolor='#2E7D32', edgecolor='none', label='Min RMSE (vs Human)'),
    # Patch(facecolor='#9AA0A6', edgecolor='none', label='Others (left)'),
    # Patch(facecolor='#C2A68C', edgecolor='none', label='Others (right)'),
    Patch(facecolor='none', edgecolor='#D76C82', linestyle='--', label='Human'),
]
ax.legend(handles=legend_handles, frameon=False, fontsize=12.5, ncol=3, loc='upper center')

plt.tight_layout()
plt.savefig('../../figure/etdrs_rmse_stacked.pdf', dpi=300)
# plt.show()

# BA

In [9]:
# def make_long_ba_df(ba_df_wide):
#     records = []
#     for _, row in ba_df_wide.iterrows():
#         diff = np.asarray(row["Diff"]).ravel()
#         avg  = np.asarray(row["Avg"]).ravel()
#         L = min(len(diff), len(avg))
#         if L == 0:
#             continue
#         # 逐样本展开
#         rec = pd.DataFrame({
#             "Model":    row["Model"],
#             "Cond":     row["Cond"],
#             "Expected_logMAR": row["Expected_logMAR"],
#             "Expected": avg[:L].astype(float),
#             "Bias":     diff[:L].astype(float),
#         })
#         records.append(rec)

#     long_df = pd.concat(records, ignore_index=True).dropna(subset=["Expected","Bias"])

#     # 每 (Model, Cond) 计算 BA 统计并回填
#     stats = (
#         long_df.groupby(["Model","Cond","Expected_logMAR"])["Bias"]
#         .agg(n="size", mean="mean", std=lambda s: np.nanstd(s, ddof=1))
#         .reset_index()
#         .rename(columns={"mean":"mean_bias","std":"std_bias"})
#     )
#     stats["LoA_Low"]  = stats["mean_bias"] - 1.96 * stats["std_bias"]
#     stats["LoA_High"] = stats["mean_bias"] + 1.96 * stats["std_bias"]
#     stats["LoA_Width"] = stats["LoA_High"] - stats["LoA_Low"]
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import numpy as np
def make_long_ba_df(ba_df_wide):
    records = []
    for i, row in ba_df_wide.iterrows():
        diff = np.asarray(row["Diff"]).ravel()
        avg  = np.asarray(row["Avg"]).ravel()
        L = min(len(diff), len(avg))
        if L == 0:
            continue
        mean_avg = avg.mean().astype(float)
        mean_diff = diff.mean().astype(float)
        # 逐样本展开
        rec = pd.DataFrame(data={
            "Model":    row["Model"],
            "Cond":     row["Cond"],
            "Expected_logMAR": row["Expected_logMAR"],
            "Expected": mean_avg,
            "Bias":     mean_diff,
            "LoA_Low":  mean_diff - 1.96 * np.std(diff, ddof=1),
            "LoA_High": mean_diff + 1.96 * np.std(diff, ddof=1),
            "LoA_Width": 2 * 1.96 * np.std(diff, ddof=1),
        },index=[i])
        records.append(rec)

    long_df = pd.concat(records, ignore_index=True).dropna(subset=["Expected","Bias"])

    # 每 (Model, Cond) 计算 BA 统计并回填
    # stats = (
    #     long_df.groupby(["Model","Cond","Expected_logMAR"])["Bias"]
    #     .agg(n="size", mean="mean", std=lambda s: np.nanstd(s, ddof=1))
    #     .reset_index()
    #     .rename(columns={"mean":"mean_bias","std":"std_bias"})
    # )
    # stats["LoA_Low"]  = stats["mean_bias"] - 1.96 * stats["std_bias"]
    # stats["LoA_High"] = stats["mean_bias"] + 1.96 * stats["std_bias"]
    # stats["LoA_Width"] = stats["LoA_High"] - stats["LoA_Low"]
    # out = long_df.merge(stats[["Model","Cond","Expected_logMAR","LoA_Low","LoA_High","LoA_Width"]], on=["Model","Cond","Expected_logMAR"], how="left")
    return long_df
ba_df = pd.concat(ba_li, axis=0)
ba_df['Expected'] = ba_df['Expected'] - (-0.24)
ba_df['Expected_logMAR'] = ba_df['Expected'] - 0.24
ba_df.drop(columns=['Expected'], inplace=True)
ba_df["LoA_Width"] = ba_df["LoA_High"] - ba_df["LoA_Low"]
ba_df['Model'] = ba_df['Model'].map(model_name_dict).fillna(ba_df['Model'])
ba_df = pd.merge(ba_df.groupby(['Model','Cond','Expected_logMAR'])['Diff'].apply(lambda x: np.concatenate(x.dropna().values,axis=0)).reset_index(), 
ba_df.groupby(['Model','Cond','Expected_logMAR'])['Avg'].apply(lambda x: np.concatenate(x.dropna().values,axis=0)).reset_index(), on=['Model','Cond','Expected_logMAR'])
# ========= 展开并计算 LoA =========
ba_df = make_long_ba_df(ba_df)   # 注意：这里覆盖变量名，后续绘图直接用 ba_df


/cis/home/qgao14/my_documents/VIOCR_infer_models/py39/lib/python3.9/site-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/cis/home/qgao14/my_documents/VIOCR_infer_models/py39/lib/python3.9/site-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

# =====================================================
# 0) 将你的 ba_df(含 list) 展开为逐样本，同时计算 LoA
#     输入列:  Model, Cond, Diff(list), Avg(list)
#     输出列:  Model, Cond, Expected, Bias, LoA_Low, LoA_High, LoA_Width
# =====================================================




# =====================================================
# 1) 你的 marker / 颜色 映射（原样保留）
# =====================================================
def get_marker(cond):
    if cond == 'Horizontal':
        return 'o'  # hollow square (注释来自你原代码，这里沿用; 实际是圆点标记)
    elif cond == 'Vertical':
        return '^'  # hollow triangle
    elif cond == 'Original':
        return 'x'  # 
    else:
        return 's'  # hollow circle

# models = ba_df['Model'].unique()

unique_conditions = ba_df['Cond'].unique()
colors = ['black', 'orange', 'red', '#6A9C89']  # 你给定的颜色序列
# 若条件数超过颜色数，做一个稳妥的扩展
if len(unique_conditions) > len(colors):
    extra = sns.color_palette("tab10", len(unique_conditions) - len(colors))
    colors = list(colors) + list(extra)
color_mapping = {cond: color for cond, color in zip(unique_conditions, colors)}

# =====================================================
# 2) 数据准备（与你的代码一致）
# =====================================================
try:
    ba_df["_ExpectedNum_"] = pd.to_numeric(ba_df["Expected"], errors="raise")
    x_col = "_ExpectedNum_"
    x_tick_labels_from = "Expected"
except Exception:
    x_col = "Expected"
    x_tick_labels_from = "Expected"

cond_list = list(ba_df["Cond"].dropna().unique())
palette = color_mapping

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

# =====================================================
# 3) FacetGrid（与你的结构一致；仅把 marker 接入）
# =====================================================
g = sns.FacetGrid(
    ba_df.sort_values(["Model", x_col]),
    col="Model",
    col_wrap=5,
    height=4.0,
    sharey=True,
    margin_titles=True,
)

def plot_ba_by_cond(data, color=None, **kws):
    ax = plt.gca()
    data = data.sort_values([x_col, "Cond"])
    x_min = -.15
    x_max = 1.8
    mu = float(np.nanmean(data["Bias"]))
    sd = float(np.nanstd(data["Bias"], ddof=1)) if data["Bias"].size > 1 else 0.0
    lo, hi = mu - 1.96*sd, mu + 1.96*sd
    ax.hlines([mu], x_min, x_max, colors='#C96868', linestyles="-",  linewidth=1.6)
    ax.hlines([lo, hi], x_min, x_max, colors='#C96868', linestyles=":", linewidth=1.2)
    for cond in cond_list:
        d = data[data["Cond"] == cond]
        if d.empty:
            continue
        c = palette[cond]
        m = get_marker(cond)  # <<<<<< 接入你的 marker 规则

        # # LoA 阴影带（本实现中 LoA 对应 (Model,Cond) 常数，填充为平行带）
        # ax.fill_between(
        #     d[x_col],
        #     d["LoA_Low"],
        #     d["LoA_High"],
        #     alpha=0.14, color=c, linewidth=0
        # )
        mu = float(np.nanmean(d["Bias"]))
        sd = float(np.nanstd(d["Bias"], ddof=1)) if d["Bias"].size > 1 else 0.0
        lo, hi = mu - 1.96*sd, mu + 1.96*sd
        # if cond != 'Original':
        #     ax.hlines([mu], x_min, x_max, colors=c, linestyles="-",  linewidth=1.6)
        #     ax.hlines([lo, hi], x_min, x_max, colors=c, linestyles=":", linewidth=1.2)
        # Bias 折线 + 散点（marker 按照你的规则）
        ax.scatter(d[x_col], d["Bias"], marker=m, color=c, label=cond, zorder=3, s=38)
        # ax.scatter(d[x_col], d["Bias"], s=15, color=c, edgecolor="white", linewidth=1., zorder=4, marker=m)



    # y=0 基线
    ax.hlines([0], x_min, x_max, linestyle="--", linewidth=1, color="0.35", zorder=1)

    # ——统计面板（与你的注释逻辑一致）——
    stats = (data
             .groupby("Cond")
             .agg(n=("Bias","size"),
                  mean_bias=("Bias","mean"),
                  std_bias=("Bias","std"),
                  mean_width=("LoA_Width","mean"))
             .reset_index())
    stats["se"] = stats["std_bias"] / np.sqrt(stats["n"]).replace(0, np.nan)
    stats["ci95"] = 1.96 * stats["se"]
    # stats = stats.sort_values("mean_bias", key=lambda s: s.abs(), ascending=False)
    stats = stats.sort_values("Cond", ascending=False)
    y_pos = 0.22
    line_height = 0.08
    for _, r in stats.iterrows():
        if r["n"] <= 1:
            continue
        cond = r["Cond"]
        c = palette.get(cond, "black")
        ci_text = f" ±{r['ci95']:.2f}" if np.isfinite(r["ci95"]) else ""
        line = f"{cond}: μBias={r['mean_bias']:.2f}{ci_text}"
        ax.text(
            0.04, y_pos, line,
            transform=ax.transAxes, va="top", ha="left",
            fontsize=12, color=c, 
        )
        y_pos -= line_height

    # 轴与网格
    ax.set_xlabel("Avg Performance")
    ax.set_ylabel("Bias (Model - Human)")
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    ax.set_ylim(-1.5, 1)
    ax.grid(alpha=0.25)

    # 若 Expected 是数值列，刻度格式化
    if x_col == "_ExpectedNum_":
        ticks = np.linspace(data[x_col].min(), data[x_col].max(), num=5)
        ax.set_xticks(ticks)
        ax.set_xticklabels([f"{t:.2f}" for t in ticks], rotation=30, ha="right")

g.map_dataframe(plot_ba_by_cond)

g.set_titles(col_template="{col_name}")

# 统一图例
handles, labels = g.axes.flat[0].get_legend_handles_labels()
uniq = dict(zip(labels, handles))
g.fig.legend(
    uniq.values(), uniq.keys(),
    loc="upper center",
    bbox_to_anchor=(0.5, 1.08),
    ncol=min(len(uniq), 5),
    frameon=True,
)

plt.tight_layout()
sns.despine(trim=True)
# plt.show()
plt.savefig('../../figure/etdrs_ba.pdf', dpi=300)

# Archive

In [ ]:
# Create lookup dictionaries for ground truth annotations and image info by image_id.
# If there are multiple ground truth annotations per image, you might need to store a list.
gt_by_chart = {}
for info, ann in zip(img_info, gt_annotations):
    id = ann["id"]
    full = {}
    full["image_id"] = ann["image_id"]
    full["file_name"] = info["file_name"]
    full["Filter_no"] = info["Filter_no"]
    full["text"] = ann["caption"]
    gt_by_chart.setdefault(id, full)


In [ ]:
import ast
# Now iterate through the model outputs and match with ground truth and image info
full_out = {}
for output_item in model_output:
    image_id = output_item["image_id"]
    
    # Optionally, if your model output's "rec_texts" is a string representation of a list,
    # convert it to an actual list.
    rec_texts_str = output_item.get("rec_texts", "")
    try:
        rec_texts = ast.literal_eval(rec_texts_str)
    except Exception:
        rec_texts = rec_texts_str
    output_item["rec_texts"] = rec_texts

    # Retrieve corresponding ground truth annotations (if any)
    gt_matches = gt_by_chart.get(image_id, {})
    chart_no, reso, row_no = Path(gt_matches.get("file_name", "___")).stem.split("_")
    full_out[image_id] = gt_matches
    full_out[image_id]["rec_texts"] = rec_texts
    full_out[image_id]["chart_no"] = chart_no
    full_out[image_id]["reso"] = reso
    full_out[image_id]["row_no"] = row_no
    
    

In [ ]:
from collections import Counter
import pandas as pd

def count_matches_per_row(row):
    left_side = pd.Series(row['text'].lower().split(' '))
    right_side = pd.Series([i.lower() for i in row['rec_texts']])
    
    counter_left = Counter(left_side.dropna()) # dic
    counter_right = Counter(right_side.dropna())
    
    # 计算两侧相同元素的个数，确保每个元素只计算一次
    matches = sum((counter_left & counter_right).values())
    total = sum(counter_left.values())
    
    # char_level
    counter_left_char = Counter(''.join(left_side.dropna().tolist()))
    counter_right_char = Counter(''.join(right_side.dropna().tolist()))
    matches_char = sum((counter_left_char & counter_right_char).values())
    total_char = sum(counter_left_char.values())
    return [matches, total - matches, total_char-matches_char]


In [ ]:

df = pd.DataFrame(full_out.values())
df = pd.merge(df, filter_df, on='Filter_no', how='inner')
# df['text'].str.split(" ", expand=True)
# # Split the 'text' column into separate columns using space as the delimiter
# split_text = df['text'].str.split(" ", expand=True)

# # Rename the split columns to L1, L2, ..., Ln
# split_text.columns = [f"L{i+1}" for i in range(split_text.shape[1])]

# # Concatenate the new columns back to the original DataFrame
# df = pd.concat([df, split_text], axis=1)
df[['match','missing','missing_char']] = df.apply(lambda row: count_matches_per_row(row), axis=1).apply(pd.Series)
df['missing_clipped'] = df['missing'].clip(upper=10)

group_sum = df.groupby(['chart_no','a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: -0.2 + x['missing_clipped'].sum()*penalty).reset_index()
group_sum.columns = ['chart_no', 'a', 'b', 'VA', 'CS', 'Cond','acuity']
group_sum = group_sum.groupby(['a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: x['acuity'].mean()).reset_index()
group_sum.columns = ['a', 'b', 'VA', 'CS', 'Cond',model_name]
group_sum
# df_merged = pd.merge(df, group_sum, on='chart_no', how='left')
# df_merged = df_merged[['image_id', 'file_name', 'Filter_no', 'text', 'rec_texts', 'chart_no',
#                        'row_no', 'a', 'b', 'VA', 'CS', 'Cond', 'acuity']]
# df_merged

/tmp/ipykernel_2338998/3606988138.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_sum = df.groupby(['chart_no','a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: -0.2 + x['missing_clipped'].sum()*0.01).reset_index()
/tmp/ipykernel_2338998/3606988138.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_sum = group_sum.groupby(['a', 'b', 'VA', 'CS', 'Cond']).apply(lambda x: x['acuity'].mean()).re

,a,b,VA,CS,Cond,gpt4o
0,0.016,1.000,1.57,1.49,Horizontal,1.796667
1,0.027,0.027,1.57,0.59,Combined,1.800000
2,0.031,1.000,1.27,1.67,Horizontal,1.766667
3,0.048,0.048,1.27,0.82,Combined,1.790000
4,0.063,1.000,0.97,1.85,Horizontal,1.556667
5,0.086,0.086,0.97,1.07,Combined,1.560000
6,0.125,1.000,0.66,1.96,Horizontal,1.253333
7,0.157,0.157,0.66,1.33,Combined,1.390000
8,0.250,1.000,0.36,2.05,Horizontal,0.957500
9,0.288,0.288,0.36,1.59,Combined,1.000000
